<a href="https://colab.research.google.com/github/KhoaYakult/Personal_ChatBot_With_RAG/blob/main/X%C3%A2y_d%E1%BB%B1ng_Chatbot_c%C3%A1_nh%C3%A2n_nhanh_ch%C3%B3ng_v%E1%BB%9Bi_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q transformers==4.52.4
%pip install -q bitsandbytes==0.46.0
%pip install -q accelerate==1.7.0
%pip install -q langchain==0.3.25
%pip install -q langchainhub==0.1.21
%pip install -q langchain-chroma==0.2.4
%pip install -q langchain-experimental==0.3.4
%pip install -q langchain-community==0.3.24
%pip install -q langchain-huggingface==0.2.0
%pip install -q python-dotenv==1.1.0
%pip install -q pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.2/457.2 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 106.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebuilt 1.0.5 requires langchain-core>=1.0.0, but you have langchain-core 0.3.81 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.7 MB/s eta 0:00:00


## Các bước trước khi cài đặt mô hình LLM

### Cài đặt thư viện để sử dụng

In [ ]:
import torch

from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.pipelines import pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline

from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import \
                        ChatMessageHistory
from langchain_community.document_loaders import \
                        PyPDFLoader, TextLoader
from langchain.chains import ConversationalRetrievalChain
from langchain_experimental.text_splitter import \
                        SemanticChunker

from langchain_chroma import Chroma

from langchain_text_splitters import \
                        RecursiveCharacterTextSplitter

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

### Đọc file pdf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Chúng ta sẽ sử dụng file pdf có tên là YOLOv10_Tutorials.pdf và sử dụng PyPDFLoader để đọc file pdf này

In [ ]:
LoaderFile = PyPDFLoader
FILE_PATH = '/content/drive/MyDrive/Xây dựng ChatBot cá nhân với RAG/YOLOv10_Tutorials.pdf'
loader = LoaderFile(FILE_PATH)
documents = loader.load()

### Khởi tạo mô hình embedding

Để chuyển đổi các văn bản thành vector, ta sử dụng mô hình embedding. Chúng ta sử dụng mô hình HuggingFaceEmbeddings với mô hình
**bkai-foundation-models/vietnamese-bi-encoder**, một mô hình được huấn luyện để xử lý ngữ nghĩa tiếng Việt.

In [ ]:
embedding = HuggingFaceEmbeddings(
    model_name = "bkai-foundation-models/vietnamese-bi-encoder"
                                )

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

### Text Splitter: tách văn bản thành những đoạn có ý nghĩa.

Chúng ta chuyển sang sử dụng SemanticChunker, một kỹ thuật giúp
tách văn bản dựa trên ngữ nghĩa, thay vì chỉ dựa vào độ dài đoạn
văn.

In [ ]:
semantic_splitter = SemanticChunker(
    embeddings = embedding,                     # Chuyển vecto thành số
    buffer_size = 1,                            # Xác định số câu cần gôm lại trước khi tách
    breakpoint_threshold_type = "percentile",   # Dựa vào phân phối % mà tách ra nếu độ tương đồng của hai đoạn văn thấp hơn ngưỡng nhất định,
                                                # chúng sẽ được tách ra làm các chunk riêng biệt
    breakpoint_threshold_amount = 95,           # Nếu độ tương đồng thấp hơn 95% sẽ bị tách thành chunk mới
    min_chunk_size = 500,                       # Mỗi đoạn văn bản không quá 500 ký tự
    add_start_index = True                      # Tạo chỉ mục và đánh dấu mỗi chunk
)

In [ ]:
docs = semantic_splitter.split_documents(documents)

In [ ]:
print("Number of semantic chunks: ", len(docs))

Number of semantic chunks:  32


### Khởi tạo VectoDataBase

Ta sử dụng Chroma để khởi tạo vector database từ các documents đã được chia thành chunk và được vector hóa.

In [ ]:
vector_db = Chroma.from_documents(documents = docs,
                                  embedding = embedding)
retriever = vector_db.as_retriever()

In [ ]:
question = retriever.invoke("What is YOLO?")
print("Number of relevant documents: ", len(question))

Number of relevant documents:  4


In [ ]:
for i in range(len(question)):
  print(question[i].page_content)
  print("*"*50)


YOLOv9
YOLOv9 [9] được giới thiệu vào năm 2024 bởi Chien-Yao Wang, I-Hau Yeh, và Hong-Yuan Mark
Liao. Mô hình này cải thiện độ chính xác và tốc độ so với YOLOv8 và giới thiệu nhiều kỹ thuật
mới như Programmable Gradient Information (PGI) và Generalized Efficient Layer Aggregation
Network (GELAN). 7
**************************************************
AI VIETNAM (AIO2024) aivietnam.edu.vn
Ở phần sau, chúng ta sẽ tập trung điểm qua các phiên bản YOLO (từ v1 đến v9). II.II. YOLOv1
YOLOv1 [1] là mô hình one-stage (hoặc single-stage) real-time object detection được giới thiệu
vào năm 2016. Hình 4: Kiến trúc mô hình YOLOv1 với 24 lớp conv và 2 lớp mlp. Ảnh: [1]. - Điểm mới: YOLOv1 sử dụng một mạng neural đơn để dự đoán cả vị trí và tên class của
các object trực tiếp từ ảnh đầu vào. - Ưu điểm: Tốc độ nhanh, khả năng object detection theo thời gian thực. - Nhược điểm: Độ chính xác không cao với các object nhỏ hoặc bị che khuất. II.III. YOLOv2
YOLOv2 [2], còn được gọi là YOLO9000, được giới thiệu

## Sử dụng mô hình ngôn ngữ lớn Vicuna

In [ ]:
nf4_config = BitsAndBytesConfig(
          load_in_4bit = True,
          bnb_4bit_quant_type = "nf4",
          bnb_4bit_use_double_quant = True,
          bnb_4bit_compute_dtype = torch.bfloat16
          )

In [ ]:
model_name = "lmsys/vicuna-7b-v1.5"
model = AutoModelForCausalLM.from_pretrained(
                                            model_name,
                                            quantization_config=nf4_config,
                                            low_cpu_mem_usage=True
                                            )
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [ ]:
model_pipeline = pipeline(
                          "text-generation",
                          model = model,
                          tokenizer = tokenizer,
                          max_new_tokens = 512,
                          pad_token_id = tokenizer.eos_token_id,
                          device_map = "auto"
                          )

llm = HuggingFacePipeline(
                          pipeline = model_pipeline,
                          )

Device set to use cuda:0


In [ ]:
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
              {"context": retriever | format_docs, "question":
              RunnablePassthrough()}
              | prompt
              | llm
              | StrOutputParser()
            )

/usr/local/lib/python3.12/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
USER_QUESTION = "Object Detection là gì?"

In [ ]:
output = rag_chain.invoke(USER_QUESTION)
answer = output.split("Answer:")[1].strip()
print(answer)

YOLOv10 là một phiên bản của YOLO (You Only Look Once) được đề xuất với những thay đổi mới, bao gồm sử dụng Consistent Dual Assignments for NMS-free Training để tối ưu hóa quá trình huấn luyện và tăng độ chính xác.


In [ ]:
print(output)

Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: Object Detection là gì? 
Context: AI VIETNAM (AIO2024) aivietnam.edu.vn
II. Bài toán Object Detection và các
phiên bản YOLO đời trước
II.I. Bài toán Object Detection
Trong Computer Vision, bài toán Object Detection hướng đến xây dựng một chương trình có
thể tự động xác định vị trí và nhận diện tên (class) của các vật thể trong một bức ảnh. Tổng
hợp hai thông tin đầu ra này còn được gọi với tên là bounding box. Từ đây, ta có thể mô tả
Input/Output của một chương trình Object Detection như sau:
- Input: Một bức ảnh. - Output: Bounding box của các vật thể cần phát hiện trong ảnh. Hình 3: Minh họa Input/Output của bài toán Object Detection. Đến thời điểm hiện tại, các phương pháp sử dụng mạng Deep Learning cho thấy hiệu suất vượt
trội. Ta 